In [1]:
import tensorflow as tf
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
import cv2
import numpy as np
import os
# backspace: 8

In [25]:
classes = [x[0][-1] if x[0][-2]=='\\' else '' for x in os.walk('data2/train')][1:]
print(classes)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y']


In [3]:
#model = keras.models.load_model(r"cse4020_CNN.h5")
model = keras.models.load_model(r"cse4020_CNN2.h5")

background = None
alpha = 0.5 
ROI = {'top':100,'bottom':300,'right':150,'left':350}

In [4]:
def calc_background(frame, alpha):

    global background
    
    if background is None:
        background = frame.copy().astype("float")
        return None
    
    '''The objective is to detect active objects from the difference obtained from 
    the reference frame and the current frame (background). We keep feeding each frame to the accumulateWeighted() function,
    and the function keeps finding the averages of all frames. '''
    
    cv2.accumulateWeighted(frame, background, alpha)

In [5]:
def detect_hand(frame, threshold=25):
    global background
    
    #absolute difference between hand frame and background frame
    abdiff = cv2.absdiff(frame, background.astype("uint8")) 
    cv2.imshow("Difference", abdiff)
    
    _ , thresholded = cv2.threshold(abdiff, threshold, 255, cv2.THRESH_BINARY) #if pixel value >threshold, 
    #255 else 0 (Binary thresholding)

    # detects change in the image color and marks it as contour. Detection of hand in front of the background
    contours, hierarchy = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) 
    # hierarchy: contains information about image topology; number of elements equal to number of contours

    if len(contours) == 0:
        return None
    else:
        #sorts by contour area and finds contour with maximum area
        hand_segment_max_cont = max(contours, key=cv2.contourArea) 
        
        return (thresholded, hand_segment_max_cont)

In [48]:
cam = cv2.VideoCapture(0)
num_frames =0
while True:
    ret, frame = cam.read()

    # filpping the frame to prevent inverted image of captured frame...
    frame = cv2.flip(frame, 1)

    frame_copy = frame.copy()

    # ROI from the frame
    roi = frame[ROI['top']:ROI['bottom'], ROI['right']:ROI['left']]

    gray_frame = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    gray_frame = cv2.GaussianBlur(gray_frame, (9, 9), 0)


    if num_frames < 70:
        
        calc_background(gray_frame, alpha)
        
        cv2.putText(frame_copy, "FETCHING BACKGROUND...PLEASE WAIT", (80, 400), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2)
    
    else: 
        # segmenting the hand region
        hand = detect_hand(gray_frame)
        

        # Checking if we are able to detect the hand...
        if hand is not None:
            
            thresholded, hand_segment = hand

            # Drawing contours around hand segment
            cv2.drawContours(frame_copy, [hand_segment + (ROI['right'], ROI['top'])], -1, (255, 0, 0),1)
            
            cv2.imshow("Thesholded Hand Image", thresholded)
            
            thresholded = cv2.resize(thresholded, (128, 128))
            thresholded = cv2.cvtColor(thresholded, cv2.COLOR_GRAY2RGB)
            thresholded = np.reshape(thresholded, (1,thresholded.shape[0],thresholded.shape[1],3))
            
            pred = model.predict(thresholded)
            cv2.putText(frame_copy, classes[np.argmax(pred)], (170, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
            
    # Draw ROI on frame_copy
    cv2.rectangle(frame_copy, (ROI['left'], ROI['top']), (ROI['right'], ROI['bottom']), (255,128,0), 3)

    # incrementing the number of frames for tracking
    num_frames += 1

    # Display the frame with segmented hand
    cv2.putText(frame_copy, "DataFlair hand sign recognition_ _ _", (10, 20), cv2.FONT_ITALIC, 0.5, (51,255,51), 1)
    cv2.imshow("Sign Detection", frame_copy)


    # Close windows with Esc
    k = cv2.waitKey(1) & 0xFF

    if k == 27:
        break

# Release the camera and destroy all the windows
cam.release()
cv2.destroyAllWindows()


In [47]:
cam.release()
cv2.destroyAllWindows()

In [22]:
print('Instructions')
print("""
1. To start storing new key, press 1
    a) Let the camera accumulate the background
    b) Make gesture
    c) Once content with the output, press Enter to append the predicted character to the key being built
    d) Repeat 'b' and 'c' to add more characters in the key
    e) To delete a character, press Backspace
    f) To store key thus formed, press Escape
Press enter to continue
""")
#1 if storing key
x = input()
string = ''
cam = cv2.VideoCapture(0)
num_frames =0
while True:
    ret, frame = cam.read()

    # filpping the frame to prevent inverted image of captured frame...
    frame = cv2.flip(frame, 1)

    frame_copy = frame.copy()

    # ROI from the frame
    roi = frame[ROI['top']:ROI['bottom'], ROI['right']:ROI['left']]

    gray_frame = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    gray_frame = cv2.GaussianBlur(gray_frame, (9, 9), 0)
    cv2.putText(frame_copy, "CSE4020 J Component - Sign Language Detection & Recognition", (10, 20), cv2.FONT_ITALIC, 0.5, (51,255,51), 1)
    cv2.imshow("Sign Detection", frame_copy)
    cv2.rectangle(frame_copy, (ROI['left'], ROI['top']), (ROI['right'], ROI['bottom']), (255,128,0), 3)
    
    ele0 = cv2.waitKey(1) & 0xFF
    if ele0==27: #break loop for escape
        print("breaking loop")
        break
    element = chr(ele0)
    #element = 'A'
    num_imgs_taken = 0
    print(chr(ele0),ele0) if ele0!=255 else print(end="")
    while chr(ele0)=='1':
        ret, frame = cam.read()

        # filpping the frame to prevent inverted image of captured frame...
        frame = cv2.flip(frame, 1)

        frame_copy = frame.copy()

        # ROI from the frame
        roi = frame[ROI['top']:ROI['bottom'], ROI['right']:ROI['left']]

        gray_frame = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        gray_frame = cv2.GaussianBlur(gray_frame, (9, 9), 0)


        if num_frames < 70:

            calc_background(gray_frame, alpha)

            cv2.putText(frame_copy, "Wait while Background is being recorded", (80, 400), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2)

        else: 
            # segmenting the hand region
            hand = detect_hand(gray_frame)


            # Checking if we are able to detect the hand...
            if hand is not None:

                thresholded, hand_segment = hand

                # Drawing contours around hand segment
                cv2.drawContours(frame_copy, [hand_segment + (ROI['right'], ROI['top'])], -1, (255, 0, 0),1)

                cv2.imshow("Thesholded Hand Image", thresholded)

                thresholded = cv2.resize(thresholded, (128, 128))
                thresholded = cv2.cvtColor(thresholded, cv2.COLOR_GRAY2RGB)
                thresholded = np.reshape(thresholded, (1,thresholded.shape[0],thresholded.shape[1],3))

                pred = model.predict(thresholded)
                cv2.putText(frame_copy, classes[np.argmax(pred)], (170, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
                ele1=cv2.waitKey(1) & 0xFF
                if ele1==13: #enter key
                    string+=classes[np.argmax(pred)]
                    #print("Enter",string,end="\r")
                if ele1==8: #backspace
                    string = string[:-1]
                    #print("Backspace",string,end="\r")
        print(string,end="\r")

        # Draw ROI on frame_copy
        cv2.rectangle(frame_copy, (ROI['left'], ROI['top']), (ROI['right'], ROI['bottom']), (255,128,0), 3)

        # incrementing the number of frames for tracking
        num_frames += 1

        # Display the frame with segmented hand
        cv2.putText(frame_copy, "CSE4020 J Component - Sign Language Detection & Recognition", (10, 20), cv2.FONT_ITALIC, 0.5, (51,255,51), 1)
        cv2.imshow("Sign Detection", frame_copy)


        # Close windows with Esc
        k = cv2.waitKey(1) & 0xFF

        if k == 27:
            break
    k = cv2.waitKey(1) & 0xFF

    if k == 27:
        break

# Release the camera and destroy all the windows
cam.release()
cv2.destroyAllWindows()
    

Instructions

1. To start storing new key, press 1
    a) Let the camera accumulate the background
    b) Make gesture
    c) Once content with the output, press Enter to append the predicted character to the key being built
    d) Repeat 'b' and 'c' to add more characters in the key
    e) To delete a character, press Backspace
    f) To store key thus formed, press Escape
Press enter to continue


 13
 8
 8
 8
 8
 8
 8
 8
 8
 13
 8
 8
 13


In [10]:
cam.release()
cv2.destroyAllWindows()

In [24]:
s = "qvqqggoig"
while s!="":
    s = s[:-1]
    print(s)

qvqqggoi
qvqqggo
qvqqgg
qvqqg
qvqq
qvq
qv
q

